In [189]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import re
analyser = SentimentIntensityAnalyzer()

In [190]:
data=pd.read_csv("tshirt-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

In [191]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/flying-machine-striped-men...,0,Striped Men Round Neck Dark Blue T-Shirt,4.3,38,3,['Colour got light not as seems dark blue but ...
1,http://flipkart.com/maniac-color-block-men-rou...,1,"Color Block Men Round Neck Red, White, Black T...",4.0,14567,1866,"['Amazing Purchase!', 'its nyc i like it', 'ni..."
2,http://flipkart.com/try-solid-men-polo-neck-wh...,2,"Solid Men Polo Neck White, Black T-Shirt",3.8,51508,7410,"['nyc product & gud quality', 'Very nice dress..."
3,http://flipkart.com/rockhard-color-block-men-r...,3,"Color Block Men Round Neck Black, Orange, Grey...",3.9,808,133,"['Nice t shirt and nice color and desine', 'Am..."
4,http://flipkart.com/tripr-printed-men-hooded-n...,4,"Printed Men Hooded Neck Dark Blue, White T-Shi...",3.9,11310,1472,"['good', 'asome nice product', ""it's very nic..."


In [192]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

In [193]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
#     print(x)
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\Miloni Mittal\Anaconda3\ANACONDA\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [194]:
# data.head()

In [195]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()
# print(total_star_givers)
# print(total_review_givers)

In [196]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [197]:
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/flying-machine-striped-men...,0,Striped Men Round Neck Dark Blue T-Shirt,4.3,38,3,colour got light not as seems dark blue but af...,2.3193,0.000078
1,http://flipkart.com/maniac-color-block-men-rou...,1,"Color Block Men Round Neck Red, White, Black T...",4.0,14567,1866,amazing purchase its nyc i like it nice tshirt,1.3686,0.027996
2,http://flipkart.com/try-solid-men-polo-neck-wh...,2,"Solid Men Polo Neck White, Black T-Shirt",3.8,51508,7410,nyc product gud quality very nice dress loved it,1.0209,0.091209
3,http://flipkart.com/rockhard-color-block-men-r...,3,"Color Block Men Round Neck Black, Orange, Grey...",3.9,808,133,nice t shirt and nice color and desine amazing...,2.8571,0.002190
4,http://flipkart.com/tripr-printed-men-hooded-n...,4,"Printed Men Hooded Neck Dark Blue, White T-Shi...",3.9,11310,1472,good asome nice product its very nice product,1.2834,0.021070


In [199]:
data.to_csv('tshirts_csv_final.csv')

In [200]:
Dict_pattern = {} 
Dict_neck={}
Dict_colour={}
item_pattern={}
item_neck={}
item_colour={}
for ind in data.index:
    men_index=0
    neck_index=0
    tshirt_index=0
    d=data['desc'][ind]
    d=d.lower()
    arr=d.split()
    men_index=arr.index("men")
    if "neck" in arr:
        neck_index=arr.index("neck")
    elif "v-neck" in arr:
        neck_index=arr.index("v-neck")
    elif "collar" in arr:
        neck_index=arr.index("collar")
    elif "crew" in arr:
        neck_index=arr.index("crew")
    tshirt_index=arr.index("t-shirt")
    pattern1=arr[0:men_index]
    neck1=arr[men_index+1:neck_index+1]
    colour1=arr[neck_index+1:tshirt_index+1]
    pattern2=" ".join(pattern1)
    neck2=" ".join(neck1)
    colour2=" ".join(colour1)
    pattern2= re.sub(r'[^\w\s]', '', pattern2) 
    neck2= re.sub(r'[^\w\s]', '', neck2) 
    colour2= re.sub(r'[^\w\s]', '', colour2) 
    if pattern2 in Dict_pattern:
        Dict_pattern[pattern2]=Dict_pattern[pattern2]+1
        item_pattern[pattern2].append(ind)
    else:
        Dict_pattern[pattern2]=1
        item_pattern[pattern2]=[]
        item_pattern[pattern2].append(ind)
        
    if neck2 in Dict_neck:
        Dict_neck[neck2]=Dict_neck[neck2]+1
        item_neck[neck2].append(ind)
    else:
        Dict_neck[neck2]=1
        item_neck[neck2]=[]
        item_neck[neck2].append(ind)
        
    if colour2 in Dict_colour:
        Dict_colour[colour2]=Dict_colour[colour2]+1
        item_colour[colour2].append(ind)
    else:
        Dict_colour[colour2]=1
        item_colour[colour2]=[]
        item_colour[colour2].append(ind)


In [201]:
print_sorted= sorted(Dict_pattern, key=Dict_pattern.get, reverse=True)
for r in print_sorted:
    print(r, Dict_pattern[r])

solid 117
printed 103
color block 77
striped 64
checkered 9
typography 6
self design 5
graphic print 5
abstract 2
chevronzig zag 2
military camouflage 2
sporty 1
perfectfit solid 1
self design solid 1
perfectfit self design 1
geometric print 1


In [202]:
row=print_sorted[0:5]+print_sorted[-5:]  
import csv
with open('print_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_pattern[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['solid', 2, 5, 8, 9, 11, 14, 16, 18, 19, 31, 33, 34, 35, 44, 46, 52, 57, 59, 61, 71, 73, 74, 75, 77, 79, 83, 84, 85, 89, 92, 93, 96, 101, 114, 115, 116, 117, 123, 124, 126, 129, 132, 133, 135, 136, 140, 143, 147, 150, 151, 153, 155, 157, 160, 163, 164, 166, 172, 173, 176, 191, 194, 197, 199, 200, 204, 205, 209, 210, 212, 219, 222, 233, 234, 236, 242, 246, 250, 251, 254, 264, 268, 273, 277, 278, 279, 285, 287, 288, 290, 292, 293, 297, 299, 305, 307, 308, 312, 315, 319, 323, 324, 325, 329, 330, 334, 345, 354, 355, 363, 368, 369, 370, 371, 372, 377, 393], ['printed', 4, 10, 12, 13, 22, 23, 24, 26, 28, 30, 37, 38, 54, 55, 56, 58, 62, 67, 68, 78, 91, 94, 97, 98, 99, 100, 102, 108, 111, 113, 118, 131, 138, 139, 141, 142, 145, 146, 148, 149, 158, 159, 171, 174, 177, 178, 181, 182, 188, 189, 207, 211, 214, 215, 216, 217, 218, 221, 226, 227, 228, 237, 238, 239, 244, 245, 253, 256, 257, 260, 266, 271, 275, 276, 291, 295, 296, 298, 300, 301, 306, 314, 317, 321, 328, 333, 336, 339, 340, 346, 353

In [203]:
neck_sorted= sorted(Dict_neck, key=Dict_neck.get, reverse=True)
for r in neck_sorted:
    print(r, Dict_neck[r])

round neck 230
polo neck 88
hooded neck 32
mandarin collar 9
v neck 9
round or crew 8
 5
collared neck 4
henley neck 4
vneck 3
turtle neck 3
high neck 1
asymmetric neck 1


In [204]:
row=neck_sorted[0:5]+neck_sorted[-5:]   
import csv
with open('neck_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_neck[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['round neck', 0, 1, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 27, 28, 30, 37, 38, 39, 42, 43, 44, 45, 46, 48, 52, 53, 54, 56, 58, 60, 62, 63, 65, 67, 68, 69, 70, 76, 78, 79, 80, 82, 83, 88, 89, 90, 92, 97, 98, 99, 102, 104, 105, 107, 108, 109, 111, 118, 120, 122, 124, 125, 127, 128, 131, 132, 133, 134, 137, 138, 139, 141, 142, 143, 145, 146, 147, 148, 149, 154, 155, 156, 158, 159, 160, 162, 164, 165, 166, 168, 169, 170, 171, 172, 174, 175, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 192, 194, 195, 196, 202, 203, 204, 205, 207, 208, 209, 211, 212, 214, 216, 217, 221, 225, 226, 227, 228, 229, 232, 234, 237, 238, 239, 241, 244, 245, 246, 247, 249, 251, 253, 256, 257, 258, 259, 260, 261, 262, 263, 266, 268, 269, 272, 273, 274, 275, 276, 277, 280, 283, 286, 287, 288, 290, 294, 295, 296, 298, 300, 301, 302, 303, 306, 307, 308, 309, 310, 314, 317, 318, 319, 320, 321, 326, 327, 328, 329, 330, 331, 334, 336, 338, 339, 340, 343, 346, 348, 349, 351, 352, 353, 355, 356, 

In [205]:
colour_sorted= sorted(Dict_colour, key=Dict_colour.get, reverse=True)
for r in colour_sorted:
    print(r, Dict_colour[r])

black tshirt 43
multicolor tshirt 34
dark blue tshirt 30
blue tshirt 25
grey tshirt 22
black grey tshirt 18
red tshirt 18
green tshirt 17
white blue tshirt 15
maroon tshirt 13
dark blue red tshirt 13
white tshirt 12
red white black tshirt 10
white black tshirt 10
black pink tshirt 10
yellow tshirt 10
orange tshirt 8
red black tshirt 6
white maroon tshirt 4
dark blue white tshirt 3
green black tshirt 3
dark blue grey tshirt 3
red white blue tshirt 3
black yellow tshirt 3
blue grey tshirt 2
reversible white tshirt 2
pink tshirt 2
red white tshirt 2
white black yellow tshirt 2
blue black tshirt 2
dark blue white yellow tshirt 2
pink black tshirt 2
white blue grey tshirt 2
dark blue maroon tshirt 2
white grey tshirt 2
reversible white black yellow tshirt 2
black orange grey tshirt 1
white black grey tshirt 1
white blue maroon tshirt 1
red blue yellow tshirt 1
blue yellow tshirt 1
maroon blue tshirt 1
purple tshirt 1
maroon black tshirt 1
white blue yellow tshirt 1
reversible blue tshirt 1


In [206]:
row=colour_sorted[0:5]+colour_sorted[-5:]
with open('colour_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
        sub = el.split(', ') 
        res.append(sub) 

    for i in range(len(res)):
        res[i]=res[i]+item_colour[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['black tshirt', 19, 33, 73, 80, 111, 113, 116, 135, 136, 140, 141, 143, 146, 157, 164, 174, 176, 187, 191, 199, 201, 213, 215, 222, 224, 229, 239, 271, 275, 293, 308, 315, 317, 321, 334, 337, 339, 345, 362, 365, 384, 387, 393], ['multicolor tshirt', 10, 17, 30, 36, 50, 64, 66, 81, 97, 100, 131, 139, 166, 170, 197, 206, 220, 230, 233, 242, 265, 267, 272, 281, 282, 284, 311, 331, 333, 335, 378, 381, 382, 383], ['dark blue tshirt', 0, 7, 25, 42, 51, 82, 94, 99, 103, 114, 122, 151, 153, 162, 192, 202, 204, 241, 253, 280, 295, 297, 309, 310, 314, 320, 327, 358, 360, 373], ['blue tshirt', 5, 34, 37, 49, 56, 57, 65, 77, 79, 110, 115, 120, 183, 193, 209, 240, 245, 252, 299, 324, 342, 354, 355, 371, 389], ['grey tshirt', 60, 71, 83, 105, 121, 124, 163, 173, 205, 219, 250, 264, 287, 289, 301, 302, 312, 323, 328, 359, 368, 374], ['dark blue black grey tshirt', 268], ['maroon black grey tshirt', 273], ['dark blue white pink tshirt', 283], ['blue beige tshirt', 298], ['men hooded black tshirt', 3